In [ ]:
def pinhole_calib (umeas, pData, camData):
    
    dx         = pData.dX
    dy         = pData.dY
    nx         = pData.nX
    ny         = pData.nY
    ncalplanes = pData.ncalplanes
    so         = camData.so  
    ncams      = camData.ncams
    
    # generate initial guesses for parameters 
    camParams   = setupCamera(camData)
    planeParams = setupPlanes(ncalplanes,z0)
    
    # generate locations of the points on each plane
    xvec        = np.arange(-(math.floor(nx/2)),math.floor(nx/2)+1)
    yvec        = np.arange(-(math.floor(ny/2)),math.floor(ny/2)+1)
    xphys       = dx*xvec
    yphys       = dy*yvec
    xmesh,ymesh = np.meshgrid(xphys,yphys)
    xmesh       = np.reshape(xmesh.T,(1,nx*ny))
    ymesh       = np.reshape(ymesh.T,(1,nx*ny))
    xy_phys     = np.concatenate((xmesh,ymesh),axis=0)
    xyz_phys    = np.concatenate((xy_phys,np.zeros([1,nX*nY])),axis=0)
    
    # initial grid to world points
    Xworld = planar_grid_to_world(planeParams,xyz_phys,pData) 
    
    # calibrate using opencv's built in calibration
    ret, camMatrix, distC, rvecs, tvecs = cv2.calibrateCamera(Xworld, umeas, Size(camData.sX, camData.sY))
    
    #notes: need world points
    #ceres uses the fact that we have multiple cameras and multiple camera parameters to refine guess of world points
    